In [2]:
# dependencies
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import pandas as pd
import qrpm_funcs as qf
from mpl_toolkits import mplot3d
from datetime import datetime
from itertools import chain
from matplotlib import cm
import scipy.stats as scistat
import scipy.optimize as sciopt

In [14]:
lastday=qf.LastYearEnd()
one_year_start_date = '2020-12-12'
start_date = '2006-12-31'

In [15]:
def get_nikkei_historical(start_date):
    
    lastday=qf.LastYearEnd()
    n225 = yf.Ticker('^N225').history(period='1d', start = start_date, end = lastday)
    n225_data = n225["Close"]
    n225_data = np.array(n225_data)

    previous = n225_data[0]
    n225_log_returns = []
    
    for i in range(1, len(n225_data)):
        n225_log_returns.append(np.log(n225_data[i]/previous))
        previous = n225_data[i]
        
    return n225_data

In [5]:
def get_fx_historical():

    lastday=qf.LastYearEnd()
    #Swiss franc, pound sterling, Japanese Yen
    seriesnames=['DEXJPUS']
    cdates,ratematrix=qf.GetFREDMatrix(seriesnames,startdate = '2007-05-12', enddate=lastday)
    multipliers=[1]

    lgdates,difflgs=qf.levels_to_log_returns(cdates,ratematrix,multipliers)

    #Mean vector and covariance matrix are inputs to efficient frontier calculations
    d=np.array(difflgs)
    #*************************End code from Chapter 4
    #vals = [i for i in difflgs]
    
    return ratematrix

fx_ret = get_fx_historical()

In [6]:
def get_equity_fx_correlation():
    
    fx_ret = np.array(get_fx_historical())
    fx_historical = fx_ret.reshape((len(fx_ret),))
    fx_historical = fx_historical[~np.isnan(fx_historical)]
    
    nikkei_historical = np.array(get_nikkei_historical(start_date))
    
    fx_historical = fx_historical[:nikkei_historical.shape[0]]
    
    # expanding the 0th dimension
    nikkei_historical = np.expand_dims(nikkei_historical,axis=0)
    fx_historical = np.expand_dims(fx_historical,axis=0)
    
    return np.corrcoef(fx_historical, nikkei_historical)[0][1]

In [7]:
# This is not working

get_equity_fx_correlation()

0.7312246022145162

In [8]:
lastday=qf.LastYearEnd()
n225 = yf.Ticker('^N225').history(period='1d', start='2016-12-31', end=lastday)
n225_data = n225["Close"]
n225_data = np.array(n225_data)

previous = n225_data[0]
n225_log_returns = []

for i in range(1, len(n225_data)):
    n225_log_returns.append(np.log(n225_data[i]/previous))
    previous = n225_data[i]
    
print(len(n225_log_returns))

1220


In [9]:
def get_spx_historical():
    
    #spx = yf.Ticker('GSPC').history(period='1d', start='2016-12-31', end='2022-12-12')
    spx = yf.download('^GSPC', period='1d', start='2016-12-31', end=lastday)
    return np.array(spx['Adj Close'])

spx = get_spx_historical()

[*********************100%***********************]  1 of 1 completed


In [10]:
def get_equity_fx_correlation():
    
    fx_ret = get_fx_historical()
    fx_historical = fx_ret.reshape((1220,))
    nikkei_historical = np.array(get_nikkei_historical())
    
    return np.corrcoef(fx_historical, nikkei_historical)

In [11]:
def get_LIBOR_3m():
    
    libor = pd.read_csv('LIBOR3m.csv')
    libor = np.array(libor['Close'])
    for i in range(len(libor)):
        libor[i] = libor[i].replace('%', '')
        libor[i] = float(libor[i])
    libor = libor.astype(float)
    libor_log_returns = []
    previous = libor[0]
    
    for i in range(1, len(libor)):
        libor_log_returns.append(np.log(libor[i]/previous))
        previous = libor[i]
        
    return libor_log_returns

In [16]:
def get_LIBOR_N225():
    
    L3m_ret = get_LIBOR_3m()
    #fx_historical = fx_ret.reshape((len(fx_ret),))
    nikkei_historical = np.array(get_nikkei_historical(one_year_start_date))
    
    return np.corrcoef(L3m_ret, nikkei_historical)[0][1]

In [17]:
get_LIBOR_N225()

-0.0462603738665728